In [ ]:
# final
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import cv2
import numpy as np
import socket
import struct
import threading
import queue
import pyaudio

# Configuration
HOST = '192.168.1.10'  # BeagleBone AI IP
VIDEO_PORT = 5000
AUDIO_PORT = 5001
SENSOR_PORT = 5002
MOTOR_AUDIO_PORT = 5003

# UI Configuration
COLORS = {
    "dark_bg": "#2a2a2a",
    "light_bg": "#3a3a3a",
    "accent": "#2196f3",
    "success": "#4caf50",
    "danger": "#f44336",
    "warning": "#ff9800",
    "text": "#ffffff"
}

class RoverClient:
    def __init__(self, root):
        self.root = root
        self.running = True
        
        # Video and audio queues
        self.video_queue = queue.Queue(maxsize=2)
        self.audio_queue = queue.Queue(maxsize=10)
        
        # Network connections
        self.motor_audio_socket = None
        
        # Initialize GUI
        self.setup_gui()
        
        # Audio setup
        self.audio = pyaudio.PyAudio()
        self.audio_stream = self.audio.open(
            format=pyaudio.paInt16,
            channels=2,
            rate=44100,
            output=True
        )
        self.input_stream = self.audio.open(
            format=pyaudio.paInt16,
            channels=2,
            rate=44100,
            input=True,
            frames_per_buffer=1024
        )
        
        # Start network connections
        self.connect_services()
        
        # Start processing threads
        threading.Thread(target=self.receive_video, daemon=True).start()
        threading.Thread(target=self.receive_audio, daemon=True).start()
        threading.Thread(target=self.receive_sensor, daemon=True).start()
        threading.Thread(target=self.play_audio, daemon=True).start()
        threading.Thread(target=self.send_audio, daemon=True).start()
        
        # Start UI updates
        self.update_video()

    def setup_gui(self):
        self.root.title("Rover Control Center")
        self.root.geometry("1280x720")
        self.root.configure(bg=COLORS["dark_bg"])
        
        # Main container
        main_frame = ttk.Frame(self.root)
        main_frame.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)
        
        # Video panel
        video_frame = ttk.Frame(main_frame)
        video_frame.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        
        self.video_label = ttk.Label(video_frame, background="black")
        self.video_label.pack(expand=True, fill=tk.BOTH)
        
        # Control panel
        control_frame = ttk.Frame(main_frame, width=300, style="Control.TFrame")
        control_frame.pack(side=tk.RIGHT, fill=tk.Y, padx=10)
        
        # Status display
        self.status_label = ttk.Label(
            control_frame,
            text="System Status: Connecting...",
            style="Status.TLabel"
        )
        self.status_label.pack(pady=10, fill=tk.X)
        
        # Motor controls
        self.create_motor_controls(control_frame)
        
        # Sensor display
        self.sensor_frame = ttk.Frame(control_frame, style="Sensor.TFrame")
        self.sensor_frame.pack(pady=20, fill=tk.X)
        
        self.sensor_icon = ttk.Label(
            self.sensor_frame,
            text="⚠",
            font=("Arial", 24),
            style="Sensor.TLabel"
        )
        self.sensor_icon.pack(side=tk.LEFT, padx=5)
        
        self.sensor_label = ttk.Label(
            self.sensor_frame,
            text="Air Quality: Initializing...",
            style="Sensor.TLabel"
        )
        self.sensor_label.pack(side=tk.LEFT)
        
        # Configure styles
        self.configure_styles()

    def configure_styles(self):
        style = ttk.Style()
        style.theme_create("custom", settings={
            "TFrame": {"configure": {"background": COLORS["dark_bg"]}},
            "TLabel": {"configure": {
                "background": COLORS["dark_bg"],
                "foreground": COLORS["text"],
                "font": ("Arial", 10)
            }},
            "Control.TFrame": {"configure": {"background": COLORS["light_bg"]}},
            "Sensor.TFrame": {"configure": {
                "background": COLORS["warning"],
                "borderwidth": 1,
                "relief": "solid"
            }},
            "Sensor.TLabel": {"configure": {
                "background": COLORS["warning"],
                "foreground": "#000000",
                "font": ("Arial", 12, "bold")
            }},
            "Status.TLabel": {"configure": {
                "background": COLORS["accent"],
                "foreground": COLORS["text"],
                "font": ("Arial", 10, "bold")
            }}
        })
        style.theme_use("custom")

    def create_motor_controls(self, parent):
        control_frame = ttk.Frame(parent)
        control_frame.pack(pady=20)
        
        btn_style = {
            "font": ("Arial", 14, "bold"),
            "width": 8,
            "relief": "flat"
        }
        
        # Forward button
        self.btn_forward = tk.Button(
            control_frame,
            text="↑",
            bg=COLORS["success"],
            command=lambda: self.send_motor_command("forward"),
            **btn_style
        )
        self.btn_forward.grid(row=0, column=1, padx=5, pady=5)
        
        # Left button
        self.btn_left = tk.Button(
            control_frame,
            text="←",
            bg=COLORS["accent"],
            command=lambda: self.send_motor_command("left"),
            **btn_style
        )
        self.btn_left.grid(row=1, column=0, padx=5, pady=5)
        
        # Stop button
        self.btn_stop = tk.Button(
            control_frame,
            text="⏹",
            bg=COLORS["danger"],
            command=lambda: self.send_motor_command("stop"),
            **btn_style
        )
        self.btn_stop.grid(row=1, column=1, padx=5, pady=5)
        
        # Right button
        self.btn_right = tk.Button(
            control_frame,
            text="→",
            bg=COLORS["accent"],
            command=lambda: self.send_motor_command("right"),
            **btn_style
        )
        self.btn_right.grid(row=1, column=2, padx=5, pady=5)
        
        # Reverse button
        self.btn_reverse = tk.Button(
            control_frame,
            text="↓",
            bg=COLORS["success"],
            command=lambda: self.send_motor_command("reverse"),
            **btn_style
        )
        self.btn_reverse.grid(row=2, column=1, padx=5, pady=5)

    def connect_services(self):
        try:
            self.motor_audio_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.motor_audio_socket.connect((HOST, MOTOR_AUDIO_PORT))
            self.update_status("Connected to motor/audio controller")
        except Exception as e:
            self.update_status(f"Connection failed: {str(e)}")

    def send_motor_command(self, command):
        try:
            if self.motor_audio_socket:
                # Send motor command with header
                header = struct.pack('!BH', 0,len(command))  # 0 for motor command
                self.motor_audio_socket.sendall(header + command.encode())
                self.update_status(f"Sent command: {command.upper()}")
        except Exception as e:
            self.update_status(f"Command failed: {str(e)}")

    def send_audio(self):
        while self.running:
            try:
                audio_data = self.input_stream.read(1024)
                if self.motor_audio_socket:
                    # Send audio with header (1 for audio data) and size
                    header = struct.pack('!BI', 1, len(audio_data))
                    self.motor_audio_socket.sendall(header + audio_data)
            except Exception as e:
                print(f"Audio send error: {str(e)}")
                break

    def receive_video(self):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            sock.connect((HOST, VIDEO_PORT))
            while self.running:
                size_data = sock.recv(4)
                if len(size_data) != 4:
                    break
                size = struct.unpack('!I', size_data)[0]
                data = b''
                while len(data) < size:
                    data += sock.recv(size - len(data))
                self.video_queue.put(data)
        finally:
            sock.close()

    def receive_audio(self):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            sock.connect((HOST, AUDIO_PORT))
            while self.running:
                size_data = sock.recv(4)
                if len(size_data) != 4:
                    break
                size = struct.unpack('!I', size_data)[0]
                data = b''
                while len(data) < size:
                    data += sock.recv(size - len(data))
                #print(data)
                self.audio_queue.put(data)
        finally:
            sock.close()

    def receive_sensor(self):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            sock.connect((HOST, SENSOR_PORT))
            while self.running:
                size_data = sock.recv(4)
                if len(size_data) != 4:
                    break
                size = struct.unpack('!I', size_data)[0]
                data = b''
                while len(data) < size:
                    data += sock.recv(size - len(data))
                #print(data)
                self.process_sensor_data(data)
        finally:
            sock.close()

    def play_audio(self):
        while self.running:
            try:
                data = self.audio_queue.get(timeout=0.1)
                self.audio_stream.write(data)
            except queue.Empty:
                pass

    def update_video(self):
        try:
            data = self.video_queue.get_nowait()
            frame = cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR)
            if frame is not None:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.config(image=imgtk)
                self.video_label.image = imgtk
        except queue.Empty:
            pass
        self.root.after(10, self.update_video)

    def process_sensor_data(self, data):
        try:
            value = data.decode()
            #print(value)
            if value == '0':
                self.sensor_label.config(text="Air Quality: NORMAL")
                self.sensor_frame.config(style="Sensor.TFrame")
                self.sensor_icon.config(text="✓", foreground="#000000")
            else:
                self.sensor_label.config(text="Air Quality: TOXIC!")
                self.sensor_frame.config(style="Danger.TFrame")
                self.sensor_icon.config(text="⚠", foreground="#ff0000")
        except:
            self.sensor_label.config(text="Sensor Error")
            self.sensor_frame.config(style="Danger.TFrame")

    def update_status(self, message):
        self.status_label.config(text=f"System Status: {message}")

    def on_close(self):
        self.running = False
        try:
            self.motor_socket.close()
            self.audio_stream.stop_stream()
            self.audio_stream.close()
            self.audio.terminate()
        except:
            pass
        self.root.destroy()

if __name__ == '__main__':
    root = tk.Tk()
    client = RoverClient(root)
    root.protocol("WM_DELETE_WINDOW", client.on_close)
    root.mainloop()